# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [84]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from apiKeys import googleKey

filePath = "outputData/citiesData.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [85]:
citiesDf = pd.read_csv(filePath)

citiesDf


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bluff,16,NZ,1607137518,73,-46.60,168.33,59.00,18.01
1,1,Barrow,90,US,1607137519,85,71.29,-156.79,10.40,18.34
2,2,Pidhorodne,90,UA,1607137521,74,48.58,35.11,28.40,15.66
3,3,Novoagansk,100,RU,1607137522,95,61.94,76.66,10.42,12.33
4,4,Busselton,29,AU,1607137392,52,-33.65,115.33,64.99,5.99
...,...,...,...,...,...,...,...,...,...,...
554,554,Waverly,90,US,1607138133,74,42.74,-84.62,37.99,4.70
555,555,Verāval,0,IN,1607138182,44,20.90,70.37,75.06,7.96
556,556,Nicolás Bravo,100,MX,1607138183,98,18.45,-88.92,70.86,2.59
557,557,Rundu,100,NaN,1607138184,83,-17.93,19.77,67.66,1.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [86]:
gmaps.configure(api_key=googleKey)

In [87]:
#putting in my locations for the heat map
coord = citiesDf[["Lat", "Lng"]]
#pulling out the humidity column for the heat map weight
hum = citiesDf["Humidity"]
#setting up my heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coord, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
#For my ideal vacation spot this time of year Im looking for somewhere where not too hold, cold, humid, or windy
idealHigh = citiesDf.loc[(citiesDf["Max Temp"] < 50)]

idealTemps = idealHigh.loc[(idealHigh["Max Temp"] > 30)]

idealHum = idealTemps.loc[(idealTemps["Humidity"] < 50)]

idealCities = idealHum.loc[(idealTemps["Wind Speed"] < 5),
                       ["City", "Country", "Lat", "Lng"]]

idealCities

,City,Country,Lat,Lng
54,Deyang,CN,31.13,104.38
142,Boulder City,US,35.98,-114.83
277,Asău,RO,46.43,26.40
314,Joshīmath,IN,30.57,79.57
320,Golden,US,39.76,-105.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [80]:
idealCities['Hotel Name'] = ""

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {  "radius": 5000,
            "type": "lodging",
            "keyword": "hotel",
            "key": googleKey}

for index, row in idealCities.iterrows():

    #getting my latitude anf longitude
    loopLat = row['Lat']
    
    loopLng = row['Lng']
    
    params['location'] = f"{loopLat},{loopLng}"

    # assemble url and make API request
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        idealCities.loc[index, 'Hotel Name'] = results[0]['name']
        

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


        
idealCities  

Closest hotel is Jinghu Hotel.
Closest hotel is Best Western Hoover Dam Hotel.
Closest hotel is Pensiunea Sophia.
Closest hotel is Blue Poppy Resorts.
Closest hotel is The Dove Inn.


,City,Country,Lat,Lng,Hotel Name
54,Deyang,CN,31.13,104.38,Jinghu Hotel
142,Boulder City,US,35.98,-114.83,Best Western Hoover Dam Hotel
277,Asău,RO,46.43,26.40,Pensiunea Sophia
314,Joshīmath,IN,30.57,79.57,Blue Poppy Resorts
320,Golden,US,39.76,-105.22,The Dove Inn


In [81]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in idealCities.iterrows()]
locations = idealCities[["Lat", "Lng"]]

In [88]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations,
                              info_box_content=hotel_info)
fig.add_layer(markerLayer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))